In [7]:
import random
import json
import pickle
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD # stochastic gradient descent optimizer
import time

In [8]:
lemmatizer = WordNetLemmatizer()
intents = json.loads(open("./intents.json").read())

words = [] # list of tokenized and lemmatized words
classes = [] # tags
documents = [] # tuple containing tokenized words and the corresponding tag
ignore_letters = ["?","!",".",","]

for intent in intents["intents"]:
    for pattern in intent["patterns"]:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent["tag"]))
        if intent["tag"] not in classes:
            classes.append(intent["tag"])
print(sorted(set(words)))
words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters] # convert each word to its base (eg. running -> run) and removes the punctuations with reference to ignore_letters
words = sorted(set(words))
print(words)
classes = sorted(set(classes))
pickle.dump(words, open("words.pkl","wb"))
pickle.dump(classes, open("classes.pkl","wb"))

["'m", "'s", '?', 'I', 'LinkedIn', 'a', 'abilities', 'about', 'absolutely', 'academic', 'afternoon', 'alright', 'am', 'ambitions', 'and', 'appreciate', 'appreciated', 'are', 'aspirations', 'at', 'background', 'bunch', 'bye', 'can', 'care', 'catch', 'competencies', 'contact', 'cool', 'cya', 'day', 'describe', 'details', 'did', 'do', 'done', 'education', 'educational', 'email', 'enjoy', 'enough', 'evening', 'examples', 'experience', 'expertise', 'fair', 'farewell', 'for', 'future', 'get', 'goals', 'going', 'good', 'goodbye', 'got', 'grateful', 'greetings', 'have', 'hello', 'help', 'hey', 'hi', 'history', 'hobbies', 'how', 'howdy', 'i', 'in', 'information', 'interests', 'is', 'it', 'job', 'jobs', 'later', 'leaving', 'like', 'list', 'lot', 'makes', 'me', 'message', 'morning', 'much', 'name', 'noted', 'of', 'ok', 'okay', 'on', 'out', 'passions', 'past', 'plans', 'previous', 'private', 'professional', 'project', 'projects', 'qualifications', 'reach', 'right', 'roger', 'see', 'sense', 'skills

### Representing the words in numerical values

In [12]:
training = []
output_empty = np.zeros(len(classes))

for document in documents:
    bag = []
    word_patterns = document[0] # contains the tokenized words
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns] # lemmatize the words
    # print(word_patterns)
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
    # print(bag)
    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append(bag + output_row)
    # print(training)
random.shuffle(training)
training = np.array(training)          

trainX = training[:, :len(words)]
trainY = training[:, len(words):]


In [5]:

model = Sequential()
model.add(Dense(128, input_shape=(len(trainX[0]),), activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(len(trainY[0]), activation='softmax'))

sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

hist = model.fit(trainX, trainY, epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print('Done')

Epoch 1/200


c:\Users\piers\PythonProjects\ChatBot\chatbot\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0740 - loss: 2.4855   
Epoch 2/200
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 775us/step - accuracy: 0.1385 - loss: 2.4579   
Epoch 3/200
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 774us/step - accuracy: 0.1835 - loss: 2.3882
Epoch 4/200
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 748us/step - accuracy: 0.2552 - loss: 2.3704
Epoch 5/200
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 769us/step - accuracy: 0.2897 - loss: 2.3409
Epoch 6/200
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 794us/step - accuracy: 0.2905 - loss: 2.2579
Epoch 7/200
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 798us/step - accuracy: 0.2958 - loss: 2.1915
Epoch 8/200
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 860us/step - accuracy: 0.3923 - loss: 2.0876
Epoch 9/200
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 940us/step - accuracy: 0.3972 - loss: 2.0803
Epoch 10/200
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 774us/step - accuracy: 0.4689 - loss: 1.9315
Epoch 11/200
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 784us/step - accuracy: 0.4392 - loss: 1.7720
Epoch 12/200
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 770us/

Done


In [21]:
lemmatizer = WordNetLemmatizer()
intents = json.loads(open("./intents.json").read())

words = pickle.load(open("words.pkl", "rb"))
classes = pickle.load(open("classes.pkl", "rb"))
model = load_model("chatbot_model.h5")

def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
    return sentence_words 

def bag_of_words(sentence):
    sentence_words = clean_up_sentence(sentence)
    bag = np.zeros(len(words))
    for sw in sentence_words:
        for i, word in enumerate(words):
            if word == sw:
                bag[i] = 1
    return bag

def predict_class(sentence):
    bow = bag_of_words(sentence)
    # convert bag of words to 2-D array and predict
    res = model.predict(np.array([bow]), verbose=0)[0]
    return {"intent":classes[np.argmax(res)], "probability":np.max(res)}

def get_response(intents_dict, intents_json):
    tag = intents_dict["intent"]
    list_of_intents = intents_json["intents"]
    print(f"Probability: {intents_dict["probability"]*100:.2f}%")
    for i in list_of_intents:
        if i["tag"] == tag:
            result = random.choice(i["responses"])
            break
    return result

print("GO! Bot is running")
while True:
    message = input("Please enter your message: ")
    ints = predict_class(message)
    res = get_response(ints, intents)
    print(res)
    if message.lower() in ["q", "quit", "bye"]:
        break
    time.sleep(1)


GO! Bot is running
Probability: 100.00%
Hey there!
Probability: 100.00%
Hi!
Probability: 100.00%
See you later!
